In [ ]:
########################################################################
# Author    : A. Alsema, J. Kotah
# Date      : October 2023
# Dataset   : Visium Spatial Transcriptomics for MS lesions, 14 WM samples
# Purpose   : post-process files using SpaceX
# Output    : RDS SpaceX analyzed object per sample that needs to be visualized and explored
# Input     : RDS SpaceX analyzed object per sample that needs post-processing
########################################################################

# note: this part of the code was run based on the SpaceX post-processing source code, located here:
# https://github.com/bayesrx/SpaceX/blob/main/R/SpaceX.R
# please also note that we ran this on a high performance cluster due to intensive RAM requirements

In [1]:
rm(list = ls())
library(SpaceX)
library(SingleCellExperiment)
library(dplyr)


Loading required package: PQLseq

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderStats,

[1] "2024-02-28 14:35:56 CET"


### Example code, ran for all samples separately

In [ ]:
#load in SpaceX processed object from Script 2 for postprocessing
fit_MSFA <- readRDS("20230707_ST33_test_spaceX_JK_not_postProcessed.rds")

#load in other SpaceX processed objects from Script 2
Gene_expression_mat = readRDS("ST33_expr_mat_test.rds")
BC_loc_test = readRDS("ST33_loc_test.rds")

In [ ]:
Spatial_locations = BC_loc_test[,1:2]
Cluster_annotations = BC_loc_test[,4]

nrun=10000 #default in SpaceX
burn=5000 #default in SpaceX

nrun = 10000 - 5000

Spatial_loc = as.data.frame(cbind(Spatial_locations,Cluster_annotations))

G <-dim(Gene_expression_mat)[2]
L <- length(unique(Spatial_loc[,3]))
Clusters <- unique(Spatial_loc[,3])

Sys.time() %>% print()
print("Start postprocessing")

SigmaPhi_post <- CorrPhi_post <- array(0, dim=c(G, G, nrun))
Sigma_l_post <- Corr_l_post <- array(0, dim=c(G, G, nrun,L))

  for(j in 1:nrun){
    SigmaPhi_post[,,j] <- tcrossprod(fit_MSFA$Posterior_samples$Phi[,,j])
    CorrPhi_post[,,j] <- cov2cor(SigmaPhi_post[,,j])
    for (l in 1:L) {
      Sigma_l_post[,,j,l] <- tcrossprod(fit_MSFA$Posterior_samples$Phi[,,j]) + tcrossprod(fit_MSFA$Posterior_samples$Lambda[[l]][,,j])
      Corr_l_post[,,j,l] <- cov2cor(Sigma_l_post[,,j,l])
    }
  }

print("Postprocessing done")
Sys.time() %>% print()

Corr_l_est <- apply(Corr_l_post, c(1,2,4), mean)     ##Shared_network :: Shared co-expression matrix
CorrPhi_est <- apply(CorrPhi_post, c(1,2), mean)  ## Cluster_network :: Cluster specific co-expression matrices

AA <- list(Posterior_samples=fit_MSFA$Posterior_samples,
           Shared_network=CorrPhi_est,
           Cluster_network=Corr_l_est, 
           Genes = colnames(Gene_expression_mat)) #add gene names from matrix colnames as this is lost

saveRDS(AA, "008_ST33_SpaceX_Post-processed.rds")

In [3]:
sessionInfo()

R version 4.1.2 (2021-11-01)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 20.04.5 LTS

Matrix products: default
BLAS/LAPACK: /data/bcn/p283607/anaconda3/envs/spacex/lib/libopenblasp-r0.3.21.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] dplyr_1.1.3                 SingleCellExperiment_1.16.0
 [3] SummarizedExperiment_1.24.0 Biobase_2.54.0             
 [5] GenomicRanges_1.46.1        GenomeInfoDb_1.30.1        
 [7] IRanges_2.28.0              S4Vectors_0.32.4           
 [9] BiocGenerics_0.40.0         MatrixGen